In [6]:
import os
from copy import deepcopy

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SplineTransformer,PolynomialFeatures,StandardScaler



from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# MODELS
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression as LR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb

In [ ]:
# COLLABORATIVE RECS -- Future application; need users

# Python --> Crab library

In [7]:
# User Input Considerations:

# I'm Feeling Lucky -> Drop a random dot in the housing map

## Price Range: Min - Max

## Property Type

## State

# Detailed

## City

## Zip Code

## Lat/Lon (Optional) # MAY REMOVE

## Property Type

## No. of Bedrooms

## No. of Bathrooms

## Square Footage

## Year Built



### NEED TO IDENTIFY WHEN DATA IS SPARSE FOR A CERTAIN AREA AND PRESENT APOLOGY MESSAGE


In [8]:
## CONSIDERATIONS

# INTRINSICALLY ASSIGN EACH PROPERTY PREDICTOR ITS LOCATION

# INTRINSICALLY ASSIGN EACH PROPERTY PREDICTOR ITS PROPERTY TYPE

# RANK THE CHOSEN NEIGHBORS IN A QUEUE, FILTER THE QUEUE BY PRICE RANGE, AND RETURN THE TOP 1-15

In [9]:
aws_bucket=os.getenv('AWS_RE_BUCKET')
aws_key=os.getenv('AWS_RE_BUCKET_KEY')
aws_secret=os.getenv('AWS_RE_BUCKET_SECRET')

date="2023-06-04"
state="CA"
states=['AL',
 'NE',
 'AK',
 'NV',
 'AZ',
 'NH',
 'AR',
 'NJ',
 'CA',
 'NM',
 'CO',
 'NY',
 'CT',
 'NC',
 'DE',
 'ND',
 'DC',
 'OH',
 'FL',
 'OK',
 'GA',
 'OR',
 'HI',
 'PA',
 'ID',
 'PR',
 'IL',
 'RI',
 'IN',
 'SC',
 'IA',
 'SD',
 'KS',
 'TN',
 'KY',
 'TX',
 'LA',
 'UT',
 'ME',
 'VT',
 'MD',
 'VA',
 'MA',
 'VI',
 'MI',
 'WA',
 'MN',
 'WV',
 'MS',
 'WI',
 'MO',
 'WY',
 'MT',
]

In [10]:
# PULLING IN DATA

df_list=[]
for st in states:
    try:
        df = pd.read_parquet(path=aws_bucket+f"/{date}/{st}.parquet",storage_options={"key":aws_key,"secret":aws_secret})
        df_list.append(df)
    except:
        pass
    
df=pd.concat(df_list)

In [11]:
df['PROPERTY TYPE'].unique()

array(['Single Family Residential', 'Vacant Land',
       'Mobile/Manufactured Home', 'Townhouse', 'Ranch',
       'Multi-Family (5+ Unit)', 'Multi-Family (2-4 Unit)', 'Condo/Co-op',
       'Parking', 'Other', 'Timeshare', 'Moorage', 'Unknown', 'None'],
      dtype=object)

In [ ]:
df_copy=deepcopy(df)

In [ ]:
def mean(list):
    return sum(list)/len(list)

def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

# PREPROCESSING
prop_types=['Single Family Residential', 'Mobile/Manufactured Home','Townhouse', 'Multi-Family (2-4 Unit)', 'Condo/Co-op','Multi-Family (5+ Unit)']


df=df.applymap(lambda x: np.nan if x=='nan' else x)

df=df[df['CITY']=='Houston'] # TEMP LIMITER 1

df=df[['CITY','STATE OR PROVINCE','PROPERTY TYPE','PRICE','BEDS','BATHS','SQUARE FEET','YEAR BUILT','DAYS ON MARKET','LATITUDE','LONGITUDE']].dropna(axis=0).reset_index(drop=True).apply(lambda row: pd.to_numeric(row,errors='ignore'))

df=df[df['PROPERTY TYPE'].isin(prop_types)]

zip_count = df[['ZIP OR POSTAL CODE']].nunique().iloc[0]

temp=pd.get_dummies(data=df['PROPERTY TYPE'])

df_full=pd.concat([df,temp],axis=1)

df_final=df_full.drop(['CITY','STATE OR PROVINCE','PROPERTY TYPE'],axis=1)

df_final=df_final[df_final['Single Family Residential']==1] # TEMP LIMITER 2
